#Recolectar y preparar datos sobre Naves Imperiales de Star Wars


In [1]:
#@title Librerías

import sys, os, re, random
import urllib, time, base64
import urllib.parse
import string
import requests
import numpy as np

from google.colab import files


print("Librerías cargadas.")

Librerías cargadas.


#Recolectar Datos de Naves Imperiales

Fuente de datos: http://insd.swcombine.com/insd.htm

Atributos disponibles:
*  Model: The ship's class, name or model number.
*  Manufacturer: The ship's manufacturer or designer.
*  Designation: The ship's combat designation or classification.
* Length: The ship's length measured in meters.
* Crew: The ship's total number of crew members - excluding droids.
* Troops: The ship's total number of troops or passengers it can carry.
* Cargo Capacity: The ship's cargo capacity measured in kilograms or metric tons.
* Consumables: How long time the ship can operate without stopping to refuel or restock stores of food, water or air.
* Hyperdrive Multiplier: The ship's main hyperdrive class rating. This measures how quickly the ship travels a hyperspace route.
* Hyperdrive Backup: The ship's backup hyperdrive class rating. This measures how quickly the ship travels a hyperspace route if its main hyperdrive is disabled.
* Speed: The ship's sublight speed rating measured in MegaLights (MGLT).
* Hull: The ship's hull/armor durability rating measured in Resistance Units (RU).
* Shields: The ship's deflector shield durability rating measured in Shield Base Durability (SBD).
* Special Features: The ship's special design features.
* Weapons: The ship's standard armament listing.
* Onboard Craft: A listing of the ship's onboard support vessels. The listing is excluding all Barges and Tugs.

In [2]:
#@title Extraer datos disponibles en la página

# funciones para manejar campos pelicula

def strbuscarEntre(text, key1, key2):
  return strbuscarEntreList(text, [key1], [key2])

def strbuscarEntreList(text, listkey1, listkey2):
 p1 = -1
 for key1 in listkey1:
  auxP1 = text.find(key1)
  if (auxP1 > -1) and ((p1 == -1) or (auxP1 < p1)):
    p1 = auxP1
 if p1 == -1:
  return ""
 else:
  p1 = p1+len(key1)
  p2 = -1
  for key2 in listkey2:
    auxP2 = text[p1:].find(key2)
    if (auxP2 > -1) and ((p2 == -1) or (auxP2 < p2)):
      p2 = auxP2
  if p2 == -1:
    return ""
  else:
    p2 = p1+p2
  return text[p1:p2]

def limpiar(text):
  if text is None:
    return ""
  else:
    text = text.replace("&quot;", " ")
    text = text.replace("&amp;", " & ")
    text = text.replace("&nbsp;", "")
    text = text.replace("\r\n", "")
    text = text.replace("  ", " ")
    text = text.replace("  ", " ")
    return text.strip()


class INSDParser:

 def __init__(self, baseURL="http://insd.swcombine.com/insd/", debug=False):
  self.__debug = debug
  self.__baseURL = baseURL
  self.__AttNames = ["Model", "Manufacturer", "Length", "Crew", "Troops", "Cargo Capacity", "Consumables", "Hyperdrive Multiplier", "Hyperdrive Backup", "Speed", "Hull", "Shields", "Special Features", "Weapons", "Onboard Craft"]
  print("\n** Parser de 'IMPERIAL NAVY SHIP DATABASE' incializado ** \n")

 def __getHTMLContent(self, url):
  return requests.get(url).text

 def extractSiteLinks(self, searchURL):
  # ejecuta y obtiene la página
  searchData = self.__getHTMLContent(searchURL)
  # extrae los links
  auxList = re.findall('<A HREF=\"([\-\_a-zA-Z0-9]+\.htm)', searchData)
  if auxList is None:
    if self.__debug:
      print("\t -- No se encuentra links de Naves!!!")
    return None
  # saca duplicados
  linksList = list(set(auxList))
  if self.__debug:
    print("\t = lista de links (", len(linksList), "): ", linksList)
  # devuelve links de obtenidos
  return linksList, searchData

 def fetchNavesInfo(self, url, shipListData):
  # carga los datos de la página
  pagedData = self.__getHTMLContent(self.__baseURL + url)
  # obtiene datos
  matchStr = strbuscarEntre(pagedData, '<BLOCKQUOTE>', '</BLOCKQUOTE>')
  if (matchStr is None) or (matchStr == ""):
    print("\t -- No se puede encontrar datos en ", url)
    return None
  # extrae datos de la página
  naveData = {}
  # agrega nombre
  naveData["Name"] = limpiar(strbuscarEntre(pagedData, '<TITLE>', '</TITLE>'))
  # extra tipo de nave de la lista
  naveData["Ship Type"] = "???"
  posName = shipListData.find(url)
  if posName > 0:
      auxData = shipListData[:posName]
      posType = auxData.rfind('<B>')
      if posType > 0:
        auxData = shipListData[posType-1:posName]
        naveData["Ship Type"] = strbuscarEntre(auxData, '<B>', '</B>')
  # extrae lista atributos
  for att in self.__AttNames:
    auxStr = strbuscarEntre(matchStr, att+":", "<")
    if auxStr != "":
      naveData[att] = auxStr
  # limpia datos extraidos
  for k in naveData.keys():
    naveData[k] = limpiar(naveData[k])
  # agrega links
  naveData["URL_data"] =  self.__baseURL + url
  naveData["URL_image"]  = self.__baseURL + 'Ship_files/' + strbuscarEntre(pagedData, '<IMG SRC=\"Ship_files/', '"')
  return naveData


 def fetchAllNavesInfo(self, url="link.htm"):
  # carga los datos de la página
  linksList, shipListData = self.extractSiteLinks( self.__baseURL + url)
  if linksList is None:
    return None
  navesList = []
  for urlNave in linksList:
    if urlNave not in ["other2.htm", "techspec.htm"]:
      # busca la información de la nave
      naveInfo = self.fetchNavesInfo(urlNave, shipListData)
      if naveInfo is not None:
        navesList.append(naveInfo)
  return navesList


mostar_detalle = True #@param {type:"boolean"}

# busca y recolecta
parser = INSDParser(debug=mostar_detalle)
navesList = parser.fetchAllNavesInfo()

# muestra
if navesList is not None:
  print("")
  print("Se recolectaron ", len(navesList), " naves.")
  print("")
  if mostar_detalle:
    for nave in navesList:
      for k in nave.keys():
        print("  " + k + " = " + nave[k])
      print("")


** Parser de 'IMPERIAL NAVY SHIP DATABASE' incializado ** 

	 = lista de links ( 109 ):  ['tieshut.htm', 'tieexm3.htm', 'tiechiss.htm', 'builder.htm', 'svelte.htm', 'interdictor.htm', 'tiehunt.htm', 'tieadvanc.htm', 'other2.htm', 'tiephantom.htm', 'assincor.htm', 'guardian.htm', 'atshut.htm', 'eclip.htm', 'gat.htm', 'tieexm5.htm', 'dominator.htm', 'tiefc.htm', 'tieln.htm', 'tiev38.htm', 'tieintrd.htm', 'nebulonb.htm', 'sfslps.htm', 'tiest.htm', 'dsd.htm', 'shadowdr.htm', 'strike.htm', 'taskcru.htm', 'asstrn.htm', 'tierpt2.htm', 'ipv1.htm', 'tiesub.htm', 'esctrn.htm', 'corvet.htm', 'tiestrcr.htm', 'tiercold.htm', 'tiescimi.htm', 'vsd1.htm', 'tiegt.htm', 'tiedefe.htm', 'tiex1.htm', 'carrack.htm', 'deltrn2.htm', 'impcusve.htm', 'i7howlrun.htm', 'lancer.htm', 'adzclass.htm', 'tierpt.htm', 'assshu2.htm', 'dungeon.htm', 'sentinel.htm', 'lawclass.htm', 'isd1.htm', 'tiebt.htm', 'tiehbomb.htm', 'tierc.htm', 'dread.htm', 'isd2.htm', 'stargal.htm', 'tiefighter.htm', 'tiebomber.htm', 'tiedroid.ht

#Revisar Datos de Naves Imperiales

In [3]:
#@title Crear un DataFrame de los datos recolectados
import pandas as pd

df = pd.DataFrame.from_dict(navesList)

display( df )

,Name,Ship Type,Model,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,Weapons,URL_data,URL_image,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft
0,TIE Shuttle Craft,TIE Support Craft,TIE/sh Shuttle,Sienar Fleet Systems,7.8 meters,1,2 (priority personnel),17 + 1 metric tons (passenger pod + cargo space),2 days,90 MGLT,28 RU,1 Laser Cannon,http://insd.swcombine.com/insd/tieshut.htm,http://insd.swcombine.com/insd/Ship_files/Ties...,NaN,NaN,NaN,NaN,NaN
1,TIE Experimental M3,TIE Experimental Craft,TIE/e3 Fighter,Sienar Fleet Systems / Imperial Department of ...,6.6 meters,1,NaN,75 kilograms,2 days,102 MGLT,16 RU,4 Laser Cannons and and 2 Concussion Missile L...,http://insd.swcombine.com/insd/tieexm3.htm,http://insd.swcombine.com/insd/Ship_files/TIEE...,NaN,NaN,NaN,NaN,NaN
2,Nssis Class Chiss Clawcraft,Other Starfighters,Nssis Class Clawcraft,Sienar Fleet Systems / Chiss Ascendancy,7.65 meters,1,NaN,25 kilograms,5 days,92 MGLT,18 RU,4 Light Laser Cannons and 4 Medium Laser Cannons.,http://insd.swcombine.com/insd/tiechiss.htm,http://insd.swcombine.com/insd/Ship_files/TIEc...,x1.5,20 SBD,NaN,NaN,NaN
3,Builder Shuttle,Landing Craft,Builder Shuttle Mark 1 and Mark 2,Cygnus Spaceworks,40 meters,4,NaN,15 metric tons (Mark 2 can hold 400 metric tons),1 week,35 MGLT,45 RU,NaN,http://insd.swcombine.com/insd/builder.htm,http://insd.swcombine.com/insd/Ship_files/BUIL...,none (Mark 2 has x1),NaN,none (Mark 2 has x10),4 Adjustable electromagnetic lifter arms.,NaN
4,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,Silviut Corporation,17 meters,6,15,50 metric tons,2 months,60 MGLT,53 RU,2 Laser Cannons and 2 Double Blaster Cannons.,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVEL...,x1,100 SBD,x5,Adjustable stabilizer fins.,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Acclamator Class Transport Ship,Heavy Ships,Acclamator Class Transport,Rothana Heavy Engineering,752 meters,700,"16,000","15,000 metric tons",1 year,8 MGLT,"1,240 RU","12 Quad Turbolaser Turrets, 24 Laser Cannons a...",http://insd.swcombine.com/insd/Acclamator.htm,http://insd.swcombine.com/insd/Ship_files/Accl...,x0.6,"2,720 SBD",x20,Planetary Flight Capability,72 TIE Series Fighters and 4 Light Transports.
103,Imperial Shuttle Pod,Other Starfighters,ISP-6,Cygnus Spaceworks,4 meters,1,NaN,100 kilograms,1 week,70 MGLT,5 RU,2 Blaster Cannons,http://insd.swcombine.com/insd/isp.htm,http://insd.swcombine.com/insd/Ship_files/ISP6...,NaN,5 SBD,NaN,Adjustable stabilizer fins.,NaN
104,Gamma Class Assault Shuttle,Shuttles,Gamma Class Assault Shuttle,Telgorn Corporation,29 meters,5,40 (Zero-G Assault Stormtroopers),5 metric tons,1 week,67 MGLT,85 RU,"4 Laser Cannon Turrets, 1 Concussion Missile L...",http://insd.swcombine.com/insd/assshut.htm,http://insd.swcombine.com/insd/Ship_files/ASHU...,x2,180 SBD,x18,NaN,NaN
105,Troop Drop-Ship,Landing Craft,MT/191 Drop-Ship,Meller & Dax,52.5 meters,1,160,200 metric tons,1 day,55 MGLT (100 MGLT during drop),45 RU (90 RU during drop),1 Light Laser Cannon,http://insd.swcombine.com/insd/dropship.htm,http://insd.swcombine.com/insd/Ship_files/TROO...,NaN,12 SBD,NaN,NaN,NaN


In [4]:
#@title Mostrar Estadísticas de datos recolectados


# variables auxiliares
atributo_clase = ""

# configura para que muestre todas las columnas y filas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# devuelve listas de columnas numéricas y no numéricas
def devolNombreColumnas(ndf):
  colValues = []
  colNoValues = []
  for col in ndf.columns:
    if ndf[col].dtypes in ("object", "bool"):
      colNoValues.append( col )
    else:
      colValues.append( col )
  return colValues, colNoValues

# función auxiliar para separar datos de entrada y de salida
def separarDatosXY(ndf, atributo_clase="", xSoloNros=True):
  # hace una copia auxiliar del data frame
  cdf = ndf.copy()
  # saca el atributo clase (OPCIONAL)
  if atributo_clase == "":
    Y = []
  else:
    # datos atributo clase
    Y = np.array( cdf.pop(atributo_clase).fillna("-NAN-") )
  if xSoloNros:
    # se queda sólo con los atributos numéricos (OPCIONAL)
    for col in cdf.columns:
      if cdf[col].dtypes == "object":
          cdf.pop( col )
  # datos de entrada
  X = np.array(cdf.fillna(0.001))
  return X, Y, np.array(cdf.columns)

def convColsNumericas(ndf, atributos_no_convertir = []):
  # hace una copia auxiliar del data frame
  cdf = ndf.copy()
  # convierte todas las no numéricas a numéricas (OPCIONAL)
  for col in cdf.columns:
    if col not in atributos_no_convertir:
      if cdf[col].dtypes == "object":
        # genera diccionario de valores
        valores = cdf[col].unique()
        diccValores = dict(zip(valores, range(len(valores))))
        # realiza el reemplazo
        cdf[col] = cdf[col].map(lambda s: diccValores.get(s) if s in diccValores else s)
  return cdf

# función auxiliar
def generar_estadisticas_detalladas(orDF, titulo=""):
  # título
  print("\n", titulo, ": ")
  # obtiene las estadísticas generales
  estDF = orDF.describe().transpose()
  #  genera y formatea las estadísticas
  if "min" in estDF and "max" in estDF:
    rangoValores = "[ " + estDF["min"].apply('{:.2f}'.format) + " ; " + estDF["max"].apply('{:.2f}'.format) + " ]"
  else:
    rangoValores = estDF["unique"].fillna(0.0).apply('{:.0f}'.format)
  rangoValores.name = "Rango Valores"
  # para campos no numéricos muestra las cantidades por valor
  for col in orDF.columns:
    if orDF[col].dtypes in ("object", "bool"):
      auxStr = str( orDF[col].value_counts() ).replace("\n", " ; ")
      if (auxStr.index("Name")-3) > 0:
        # saca lo del final porque no sirve
        auxStr = auxStr[:auxStr.index("Name")-3]
      rangoValores[col] = "{ " + auxStr + " }"
  if "mean" in estDF and "std" in estDF:
    promValores = estDF["mean"].fillna(0.0).apply('{:.3f}'.format) + " ± " + estDF["std"].fillna(0.0).apply('{:.3f}'.format)
  else:
    promValores = estDF["count"].apply('{:.0f}'.format)
  promValores.name = "Promedio ± Desvío"
  # obtiene valores "ceros" y nulos
  zero_val = (orDF == 0.00).astype(int).sum(axis=0)
  zero_val.name = "¿Valores Ceros?"
  mis_val = orDF.isnull().sum()
  mis_val.name = "¿Valores Nulos?"
  # prepara la nueva tabla para mostrar
  nTable = pd.concat([orDF.dtypes, rangoValores, promValores, zero_val, mis_val], axis=1)
  nTable = nTable.rename( columns = {0: 'Tipo Valor',  1: 'Rango Valores', 2: 'Promedio ± Desvío', 3: '¿Valores Ceros?', 4: '¿Valores Nulos?' } )
  # muestra la nueva tabla
  pd.set_option('max_colwidth', None)
  display(nTable.fillna("-"))
  print("Tiene " + str(orDF.shape[1]) + " atributos y " + str(orDF.shape[0]) + " ejemplos.")
  print("\n")
  return

# muestra las estadísticas
generar_estadisticas_detalladas(df, "> Estadísticas de los datos recolectados")



 > Estadísticas de los datos recolectados : 


<ipython-input-4-22854d31a8c4>:65: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rangoValores = estDF["unique"].fillna(0.0).apply('{:.0f}'.format)


Tipo Valor  \
Name                      object   
Ship Type                 object   
Model                     object   
Manufacturer              object   
Length                    object   
Crew                      object   
Troops                    object   
Cargo Capacity            object   
Consumables               object   
Speed                     object   
Hull                      object   
Weapons                   object   
URL_data                  object   
URL_image                 object   
Hyperdrive Multiplier     object   
Shields                   object   
Hyperdrive Backup         object   
Special Features          object   
Onboard Craft             object   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Tiene 19 atributos y 107 ejemplos.




#Preparar Datos de Naves Imperiales

In [5]:
#@title Preparar Campos donde se extrae parte numérica eliminando texto

# Hace copia de trabajo (para romper datos extraidos)
ndf = df.copy()

# funciones auxiliares
def extraerPrimerNro(columnName):
  global ndf
  # siempre elimina "," como separador de miles
  reemplazar(columnName, ",")
  # toma número con decimales
  ndf[columnName] = ndf[columnName].str.extract('(\d+\.?\d*)\D*', expand=True)
  return

def reemplazar(columnName, oldValue, newValue=""):
  global ndf
  ndf[columnName] = ndf[columnName].str.replace(oldValue, newValue, regex=False)
  ndf[columnName] = ndf[columnName].fillna(0)
  return

def completarVacios(columnName, newValue=-1):
  global ndf
  ndf[columnName] = ndf[columnName].fillna(newValue)
  return

def convertirFloat(columnName):
  global ndf
  ndf[columnName] = ndf[columnName].astype(float)

print("-procesa Length")
reemplazar("Length", " meters")
extraerPrimerNro("Length")
completarVacios("Length", 0)
convertirFloat("Length")

print("-procesa Crew")
extraerPrimerNro("Crew")
completarVacios("Crew", 0)
convertirFloat("Crew")

print("-procesa Troops")
extraerPrimerNro("Troops")
completarVacios("Troops", 0)
convertirFloat("Troops")

print("-procesa Hyperdrive Multiplier")
extraerPrimerNro("Hyperdrive Multiplier")
completarVacios("Hyperdrive Multiplier", -1)
convertirFloat("Hyperdrive Multiplier")

print("-procesa Hyperdrive Backup")
extraerPrimerNro("Hyperdrive Backup")
completarVacios("Hyperdrive Backup", -1)
convertirFloat("Hyperdrive Backup")

print("-procesa Speed")
reemplazar("Speed", " MGLT")
extraerPrimerNro("Speed")
completarVacios("Speed", 0)
convertirFloat("Speed")

print("-procesa Hull")
reemplazar("Hull", " RU")
extraerPrimerNro("Hull")
completarVacios("Hull", 0)
convertirFloat("Hull")

print("-procesa Shields")
reemplazar("Shields", " SBD")
extraerPrimerNro("Shields")
completarVacios("Shields", -1)
convertirFloat("Shields")

# muestra cambios
ndf.head()

-procesa Length
-procesa Crew
-procesa Troops
-procesa Hyperdrive Multiplier
-procesa Hyperdrive Backup
-procesa Speed
-procesa Hull
-procesa Shields


,Name,Ship Type,Model,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,Weapons,URL_data,URL_image,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft
0,TIE Shuttle Craft,TIE Support Craft,TIE/sh Shuttle,Sienar Fleet Systems,7.80,1.0,2.0,17 + 1 metric tons (passenger pod + cargo space),2 days,90.0,28.0,1 Laser Cannon,http://insd.swcombine.com/insd/tieshut.htm,http://insd.swcombine.com/insd/Ship_files/Tieshut.jpg,-1.0,-1.0,-1.0,NaN,NaN
1,TIE Experimental M3,TIE Experimental Craft,TIE/e3 Fighter,Sienar Fleet Systems / Imperial Department of Military Research,6.60,1.0,0.0,75 kilograms,2 days,102.0,16.0,4 Laser Cannons and and 2 Concussion Missile Launchers.,http://insd.swcombine.com/insd/tieexm3.htm,http://insd.swcombine.com/insd/Ship_files/TIEEXM3.JPG,-1.0,-1.0,-1.0,NaN,NaN
2,Nssis Class Chiss Clawcraft,Other Starfighters,Nssis Class Clawcraft,Sienar Fleet Systems / Chiss Ascendancy,7.65,1.0,0.0,25 kilograms,5 days,92.0,18.0,4 Light Laser Cannons and 4 Medium Laser Cannons.,http://insd.swcombine.com/insd/tiechiss.htm,http://insd.swcombine.com/insd/Ship_files/TIEchiss.jpg,1.5,20.0,-1.0,NaN,NaN
3,Builder Shuttle,Landing Craft,Builder Shuttle Mark 1 and Mark 2,Cygnus Spaceworks,40.00,4.0,0.0,15 metric tons (Mark 2 can hold 400 metric tons),1 week,35.0,45.0,NaN,http://insd.swcombine.com/insd/builder.htm,http://insd.swcombine.com/insd/Ship_files/BUILDER.JPG,2.0,-1.0,2.0,4 Adjustable electromagnetic lifter arms.,NaN
4,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,Silviut Corporation,17.00,6.0,15.0,50 metric tons,2 months,60.0,53.0,2 Laser Cannons and 2 Double Blaster Cannons.,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVELTE.JPG,1.0,100.0,5.0,Adjustable stabilizer fins.,NaN


In [6]:
#@title Preparar campo Cargo Capacity

print("-procesa Cargo Capacity (pasa todo a kilogramos)")

# realiza los cambios considerando tipo de métrica
auxOriList = list(df["Cargo Capacity"])
auxNewList = []
for val in auxOriList:
  #print(val)
  if (val is None) or (str(val) in ["nan", "(varies according to mission profile)"]):
    auxNewList.append( -1 )
  elif str(val)=="Mu-1 and Mu-2: 100 metric tons / Mu-3: 50 metric tons":
    auxNewList.append( 75 )
  else:
    val = val.replace(",", "")
    val = val.replace(" + 1", "")
    if "(" in val:
      posParent = val.index("(")
    else:
      posParent = -1
    if "-" in val:
      # calcula promedio
      posSep1 = val.index("-")
      posSep2 = val.index(" ")
      nval = ( float(val[0:posSep1]) + float(val[posSep1+1:posSep2]) ) / 2
      val = str(nval) + val[posSep2:]
    if "or" in val:
      # calcula promedio
      posSep1 = val.index(" or ")
      posSep2 = val[posSep1+4:].index(" ") + posSep1 + 4
      nval = ( float(val[0:posSep1]) + float(val[posSep1+4:posSep2]) ) / 2
      val = str(nval) + val[posSep2:]
    if "kilograms" in val:
      posAux = val.index("kilograms")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) )
    elif "metric ton" in val:
      posAux = val.index("metric ton")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) * 1000 )
    else:
      print("No se puede procesar: ", val)

# actualiza los datos
ndf["Cargo Capacity"] = auxNewList
convertirFloat("Cargo Capacity")


ndf.head()

-procesa Cargo Capacity (pasa todo a kilogramos)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,Weapons,URL_data,URL_image,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft
0,TIE Shuttle Craft,TIE Support Craft,TIE/sh Shuttle,Sienar Fleet Systems,7.80,1.0,2.0,17000.0,2 days,90.0,28.0,1 Laser Cannon,http://insd.swcombine.com/insd/tieshut.htm,http://insd.swcombine.com/insd/Ship_files/Tieshut.jpg,-1.0,-1.0,-1.0,NaN,NaN
1,TIE Experimental M3,TIE Experimental Craft,TIE/e3 Fighter,Sienar Fleet Systems / Imperial Department of Military Research,6.60,1.0,0.0,75.0,2 days,102.0,16.0,4 Laser Cannons and and 2 Concussion Missile Launchers.,http://insd.swcombine.com/insd/tieexm3.htm,http://insd.swcombine.com/insd/Ship_files/TIEEXM3.JPG,-1.0,-1.0,-1.0,NaN,NaN
2,Nssis Class Chiss Clawcraft,Other Starfighters,Nssis Class Clawcraft,Sienar Fleet Systems / Chiss Ascendancy,7.65,1.0,0.0,25.0,5 days,92.0,18.0,4 Light Laser Cannons and 4 Medium Laser Cannons.,http://insd.swcombine.com/insd/tiechiss.htm,http://insd.swcombine.com/insd/Ship_files/TIEchiss.jpg,1.5,20.0,-1.0,NaN,NaN
3,Builder Shuttle,Landing Craft,Builder Shuttle Mark 1 and Mark 2,Cygnus Spaceworks,40.00,4.0,0.0,15000.0,1 week,35.0,45.0,NaN,http://insd.swcombine.com/insd/builder.htm,http://insd.swcombine.com/insd/Ship_files/BUILDER.JPG,2.0,-1.0,2.0,4 Adjustable electromagnetic lifter arms.,NaN
4,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,Silviut Corporation,17.00,6.0,15.0,50000.0,2 months,60.0,53.0,2 Laser Cannons and 2 Double Blaster Cannons.,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVELTE.JPG,1.0,100.0,5.0,Adjustable stabilizer fins.,NaN


In [7]:
#@title Preparar campo Consumables

print("-procesa Consumables (pasa todo a meses)")

# realiza los cambios considerando tipo de métrica
auxOriList = list(df["Consumables"])
auxNewList = []
for val in auxOriList:
  #print(val)
  if (val is None) or (str(val) in ["nan", "(varies according to mission profile)"]):
    auxNewList.append( -1 )
  if str(val) == "Mu-1 and Mu-2: 6 months / Mu-3: 2 months":
    auxNewList.append( 4 )
  else:
    val = val.replace(",", "")
    #val = val.replace(" + 1", "")
    if "(" in val:
      posParent = val.index("(")
    else:
      posParent = -1
    if "-" in val:
      # calcula promedio
      posSep1 = val.index("-")
      posSep2 = val.index(" ")
      nval = ( float(val[0:posSep1]) + float(val[posSep1+1:posSep2]) ) / 2
      val = str(nval) + val[posSep2:]
    if "or" in val:
      # calcula promedio
      posSep1 = val.index(" or ")
      posSep2 = val[posSep1+4:].index(" ") + posSep1 + 4
      nval = ( float(val[0:posSep1]) + float(val[posSep1+4:posSep2]) ) / 2
      val = str(nval) + val[posSep2:]
    if "month" in val:
      posAux = val.index("month")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) )
    elif "year" in val:
      posAux = val.index("year")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) * 12 )
    elif "week" in val:
      posAux = val.index("week")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) / 4.5 )
    elif "day" in val:
      posAux = val.index("day")
      if (posParent>0) and (posParent<posAux):
        posAux = posParent
      auxNewList.append( float(val[:posAux].strip()) / 30 )
    else:
      print("No se puede procesar: ", val)

# actualiza los datos
ndf["Consumables"] = auxNewList
convertirFloat("Consumables")

ndf.head()

-procesa Consumables (pasa todo a meses)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,Weapons,URL_data,URL_image,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft
0,TIE Shuttle Craft,TIE Support Craft,TIE/sh Shuttle,Sienar Fleet Systems,7.80,1.0,2.0,17000.0,0.066667,90.0,28.0,1 Laser Cannon,http://insd.swcombine.com/insd/tieshut.htm,http://insd.swcombine.com/insd/Ship_files/Tieshut.jpg,-1.0,-1.0,-1.0,NaN,NaN
1,TIE Experimental M3,TIE Experimental Craft,TIE/e3 Fighter,Sienar Fleet Systems / Imperial Department of Military Research,6.60,1.0,0.0,75.0,0.066667,102.0,16.0,4 Laser Cannons and and 2 Concussion Missile Launchers.,http://insd.swcombine.com/insd/tieexm3.htm,http://insd.swcombine.com/insd/Ship_files/TIEEXM3.JPG,-1.0,-1.0,-1.0,NaN,NaN
2,Nssis Class Chiss Clawcraft,Other Starfighters,Nssis Class Clawcraft,Sienar Fleet Systems / Chiss Ascendancy,7.65,1.0,0.0,25.0,0.166667,92.0,18.0,4 Light Laser Cannons and 4 Medium Laser Cannons.,http://insd.swcombine.com/insd/tiechiss.htm,http://insd.swcombine.com/insd/Ship_files/TIEchiss.jpg,1.5,20.0,-1.0,NaN,NaN
3,Builder Shuttle,Landing Craft,Builder Shuttle Mark 1 and Mark 2,Cygnus Spaceworks,40.00,4.0,0.0,15000.0,0.222222,35.0,45.0,NaN,http://insd.swcombine.com/insd/builder.htm,http://insd.swcombine.com/insd/Ship_files/BUILDER.JPG,2.0,-1.0,2.0,4 Adjustable electromagnetic lifter arms.,NaN
4,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,Silviut Corporation,17.00,6.0,15.0,50000.0,2.000000,60.0,53.0,2 Laser Cannons and 2 Double Blaster Cannons.,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVELTE.JPG,1.0,100.0,5.0,Adjustable stabilizer fins.,NaN


In [8]:
#@title Preparar campo Manufacturer

print("-procesa Manufacturer (asigna codigo valor ID)")

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(df["Manufacturer"])
ndf["Manufacturer"] = encoder.transform(df["Manufacturer"])
convertirFloat("Manufacturer")

# muestra códificación asignada
print("\n\tCodificación asignada: ")
for i in range(len(encoder.classes_)):
  print("\t\t", i, ":", encoder.classes_[i])
print("")

ndf.head()

-procesa Manufacturer (asigna codigo valor ID)

	Codificación asignada: 
		 0 : Byss Worx / Imperial Department of Military Research
		 1 : CPG Space Products
		 2 : Corellian Engineering Corporation
		 3 : Cygnus Spaceworks
		 4 : Cygnus Spaceworks / Sienar Fleet Systems
		 5 : Damorian Manufacturing Corporation
		 6 : Imperial Shipyards
		 7 : Incom Corporation
		 8 : Kuat Drive Yards
		 9 : Loronar
		 10 : Loronar / Rendili StarDrive /Sienar Fleet Systems and Kuat Drive Yards
		 11 : Meller & Dax
		 12 : Mesens Corporation
		 13 : Rendili StarDrive
		 14 : Republic Sienar Systems
		 15 : Rothana Heavy Engineering
		 16 : Rothana Heavy Engineering / Kuat Drive Yards
		 17 : Santhe / Sienar Technologies
		 18 : Sienar Fleet Systems
		 19 : Sienar Fleet Systems / Chiss Ascendancy
		 20 : Sienar Fleet Systems / Imperial Department of Military Research
		 21 : Sienar Fleet Systems / Shobquix Yards
		 22 : Sienar Fleet Systems / Zsinj Development Incorporated
		 23 : Sienar Fleet Systems/

,Name,Ship Type,Model,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,Weapons,URL_data,URL_image,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft
0,TIE Shuttle Craft,TIE Support Craft,TIE/sh Shuttle,18.0,7.80,1.0,2.0,17000.0,0.066667,90.0,28.0,1 Laser Cannon,http://insd.swcombine.com/insd/tieshut.htm,http://insd.swcombine.com/insd/Ship_files/Tieshut.jpg,-1.0,-1.0,-1.0,NaN,NaN
1,TIE Experimental M3,TIE Experimental Craft,TIE/e3 Fighter,20.0,6.60,1.0,0.0,75.0,0.066667,102.0,16.0,4 Laser Cannons and and 2 Concussion Missile Launchers.,http://insd.swcombine.com/insd/tieexm3.htm,http://insd.swcombine.com/insd/Ship_files/TIEEXM3.JPG,-1.0,-1.0,-1.0,NaN,NaN
2,Nssis Class Chiss Clawcraft,Other Starfighters,Nssis Class Clawcraft,19.0,7.65,1.0,0.0,25.0,0.166667,92.0,18.0,4 Light Laser Cannons and 4 Medium Laser Cannons.,http://insd.swcombine.com/insd/tiechiss.htm,http://insd.swcombine.com/insd/Ship_files/TIEchiss.jpg,1.5,20.0,-1.0,NaN,NaN
3,Builder Shuttle,Landing Craft,Builder Shuttle Mark 1 and Mark 2,3.0,40.00,4.0,0.0,15000.0,0.222222,35.0,45.0,NaN,http://insd.swcombine.com/insd/builder.htm,http://insd.swcombine.com/insd/Ship_files/BUILDER.JPG,2.0,-1.0,2.0,4 Adjustable electromagnetic lifter arms.,NaN
4,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,24.0,17.00,6.0,15.0,50000.0,2.000000,60.0,53.0,2 Laser Cannons and 2 Double Blaster Cannons.,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVELTE.JPG,1.0,100.0,5.0,Adjustable stabilizer fins.,NaN


In [9]:
#@title Preparar campo Weapons

print("-procesa Weapons (extrae lista de armas y genera nuevas columnas numéricas)")

import re

newValues = []

for f in df["Weapons"]:
  # extrae la lista de armas
  auxSplit =  re.split(", | and ", str(f))
  #print(auxSplit)
  # define columnas en base subcadenas obtenidas
  auxDict = {}
  for w in auxSplit:
    if w=="nan":
      continue
    res = re.search('(\d+\.?\d*)\s(.+)', w)
    if (res is not None):
      ##print(res.group(1), "*", res.group(2))
      auxW = str(res.group(2)).replace(".", "").upper().strip()
      if "(" in auxW:
        # saca toda aclaración entre paréntesis
        auxW = auxW[:auxW.find("(")].strip()
      # si no es plural, le agrega S (para unificar columnas)
      if auxW[len(auxW)-1] != "S":
        auxW = auxW + "S"
      auxQ = float(res.group(1))
    else:
      auxW = w.upper().strip()
      auxQ = 1.0
    if len(auxW)<1:
      auxW = "OTHER"
    if auxW in auxDict:
      auxDict[auxW] = auxDict[auxW] + auxQ
    else:
      auxDict[auxW] = auxQ

  newValues.append( auxDict )


##print(newValues)

# crea nuevo data frame
auxDF = pd.DataFrame.from_dict(newValues)
# completa valores vacios con cero
for col in auxDF:
  auxDF[col] = auxDF[col].fillna(0)
# agrega al data frame anterior y elimina columna original
ndf = pd.concat([ndf, auxDF], axis=1)
ndf = ndf.drop(["Weapons"], axis=1)

ndf.head()

-procesa Weapons (extrae lista de armas y genera nuevas columnas numéricas)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,URL_data,URL_image,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft,LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LIGHT LASER CANNONS,MEDIUM LASER CANNONS,DOUBLE BLASTER CANNONS,QUAD LASER CANNONS,GRAVITY WELL PROJECTORS,ION CANNONS,PROTON TORPEDO LAUNCHERS,GENERAL PURPOSE LAUNCHERS,DOUBLE TURBOLASER CANNONS,TURBOLASER CANNONS,TRACTOR BEAM PROJECTORS,LASER CANNON TURRETS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,SUPERLASERS,MEDIUM ION CANNONS,TURBOLASER BATTERIES,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,TWIN LASER CANNONS,QUAD BLASTER CANNON,EMP CANNONS,MEDIUM TURBOLASERS,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,BLASTER CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,HULL-CUTTING AIRLOCKS,ION CANNON BATTERIES,ION CANNON TURRETS,REPEATING BLASTER TURRETS,QUAD TURBOLASER CANNONS,HEAVY TURBOLASER CANNONS,HEAVY TURBOLASER TURRETS,TURBOLASER TURRETS,DOUBLE LASER CANNON TURRETS,QUAD LASER CANNON TURRETS,(VARIES ACCORDING TO SHIP MODEL),BUILD-IN SPACE BOMB.,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,DOUBLE BLASTER CANNON TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,HEAVY LASER CANNON TURRETS,DOUBLE LASER CANNONS,TWIN BLASTER CANNON TURRETS,LASER CANNONS TURRETS,HEAVY TRACTORBEAM PROJECTORS,HEAVY BLASTER CANNONS,DOUBLE HEAVY LASER TURRETS,QUAD TURBOLASER TURRETS
0,TIE Shuttle Craft,TIE Support Craft,TIE/sh Shuttle,18.0,7.80,1.0,2.0,17000.0,0.066667,90.0,28.0,http://insd.swcombine.com/insd/tieshut.htm,http://insd.swcombine.com/insd/Ship_files/Tieshut.jpg,-1.0,-1.0,-1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TIE Experimental M3,TIE Experimental Craft,TIE/e3 Fighter,20.0,6.60,1.0,0.0,75.0,0.066667,102.0,16.0,http://insd.swcombine.com/insd/tieexm3.htm,http://insd.swcombine.com/insd/Ship_files/TIEEXM3.JPG,-1.0,-1.0,-1.0,NaN,NaN,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Nssis Class Chiss Clawcraft,Other Starfighters,Nssis Class Clawcraft,19.0,7.65,1.0,0.0,25.0,0.166667,92.0,18.0,http://insd.swcombine.com/insd/tiechiss.htm,http://insd.swcombine.com/insd/Ship_files/TIEchiss.jpg,1.5,20.0,-1.0,NaN,NaN,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Builder Shuttle,Landing Craft,Builder Shuttle Mark 1 and Mark 2,3.0,40.00,4.0,0.0,15000.0,0.222222,35.0,45.0,http://insd.swcombine.com/insd/builder.htm,http://insd.swcombine.com/insd/Ship_files/BUILDER.JPG,2.0,-1.0,2.0,4 Adjustable electromagnetic lifter arms.,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,24.0,17.00,6.0,15.0,50000.0,2.000000,60.0,53.0,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVELTE.JPG,1.0,100.0,5.0,Adjustable stabilizer fins.,NaN,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [10]:
#@title Preparar campo Special Features

# función auxiliar para clusterizar datos
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

def clusterizarDatosString(colunmnName, cantClusters=5):
  global ndf
  ndf[colunmnName] = ndf[colunmnName].fillna("-")
  documents  = list( ndf[colunmnName] )

  vectorizer = TfidfVectorizer(stop_words='english')
  X = vectorizer.fit_transform(documents)

  model = KMeans(n_clusters=cantClusters, init='k-means++', max_iter=100, n_init=1)
  ndf[colunmnName] = model.fit_transform(X)



print("-procesa Special Features (usando clustering)")

clusterizarDatosString("Special Features")
convertirFloat("Special Features")

ndf.head()

-procesa Special Features (usando clustering)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,URL_data,URL_image,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft,LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LIGHT LASER CANNONS,MEDIUM LASER CANNONS,DOUBLE BLASTER CANNONS,QUAD LASER CANNONS,GRAVITY WELL PROJECTORS,ION CANNONS,PROTON TORPEDO LAUNCHERS,GENERAL PURPOSE LAUNCHERS,DOUBLE TURBOLASER CANNONS,TURBOLASER CANNONS,TRACTOR BEAM PROJECTORS,LASER CANNON TURRETS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,SUPERLASERS,MEDIUM ION CANNONS,TURBOLASER BATTERIES,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,TWIN LASER CANNONS,QUAD BLASTER CANNON,EMP CANNONS,MEDIUM TURBOLASERS,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,BLASTER CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,HULL-CUTTING AIRLOCKS,ION CANNON BATTERIES,ION CANNON TURRETS,REPEATING BLASTER TURRETS,QUAD TURBOLASER CANNONS,HEAVY TURBOLASER CANNONS,HEAVY TURBOLASER TURRETS,TURBOLASER TURRETS,DOUBLE LASER CANNON TURRETS,QUAD LASER CANNON TURRETS,(VARIES ACCORDING TO SHIP MODEL),BUILD-IN SPACE BOMB.,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,DOUBLE BLASTER CANNON TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,HEAVY LASER CANNON TURRETS,DOUBLE LASER CANNONS,TWIN BLASTER CANNON TURRETS,LASER CANNONS TURRETS,HEAVY TRACTORBEAM PROJECTORS,HEAVY BLASTER CANNONS,DOUBLE HEAVY LASER TURRETS,QUAD TURBOLASER TURRETS
0,TIE Shuttle Craft,TIE Support Craft,TIE/sh Shuttle,18.0,7.80,1.0,2.0,17000.0,0.066667,90.0,28.0,http://insd.swcombine.com/insd/tieshut.htm,http://insd.swcombine.com/insd/Ship_files/Tieshut.jpg,-1.0,-1.0,-1.0,0.892039,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TIE Experimental M3,TIE Experimental Craft,TIE/e3 Fighter,20.0,6.60,1.0,0.0,75.0,0.066667,102.0,16.0,http://insd.swcombine.com/insd/tieexm3.htm,http://insd.swcombine.com/insd/Ship_files/TIEEXM3.JPG,-1.0,-1.0,-1.0,0.892039,NaN,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Nssis Class Chiss Clawcraft,Other Starfighters,Nssis Class Clawcraft,19.0,7.65,1.0,0.0,25.0,0.166667,92.0,18.0,http://insd.swcombine.com/insd/tiechiss.htm,http://insd.swcombine.com/insd/Ship_files/TIEchiss.jpg,1.5,20.0,-1.0,0.892039,NaN,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Builder Shuttle,Landing Craft,Builder Shuttle Mark 1 and Mark 2,3.0,40.00,4.0,0.0,15000.0,0.222222,35.0,45.0,http://insd.swcombine.com/insd/builder.htm,http://insd.swcombine.com/insd/Ship_files/BUILDER.JPG,2.0,-1.0,2.0,1.244250,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,24.0,17.00,6.0,15.0,50000.0,2.000000,60.0,53.0,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVELTE.JPG,1.0,100.0,5.0,1.190873,NaN,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [11]:
#@title Preparar campo Onboard Craft

print("-procesa Onboard Craft (usando clustering)")

clusterizarDatosString("Onboard Craft")
convertirFloat("Onboard Craft")

ndf.head()

-procesa Onboard Craft (usando clustering)


,Name,Ship Type,Model,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,URL_data,URL_image,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft,LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LIGHT LASER CANNONS,MEDIUM LASER CANNONS,DOUBLE BLASTER CANNONS,QUAD LASER CANNONS,GRAVITY WELL PROJECTORS,ION CANNONS,PROTON TORPEDO LAUNCHERS,GENERAL PURPOSE LAUNCHERS,DOUBLE TURBOLASER CANNONS,TURBOLASER CANNONS,TRACTOR BEAM PROJECTORS,LASER CANNON TURRETS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,SUPERLASERS,MEDIUM ION CANNONS,TURBOLASER BATTERIES,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,TWIN LASER CANNONS,QUAD BLASTER CANNON,EMP CANNONS,MEDIUM TURBOLASERS,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,BLASTER CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,HULL-CUTTING AIRLOCKS,ION CANNON BATTERIES,ION CANNON TURRETS,REPEATING BLASTER TURRETS,QUAD TURBOLASER CANNONS,HEAVY TURBOLASER CANNONS,HEAVY TURBOLASER TURRETS,TURBOLASER TURRETS,DOUBLE LASER CANNON TURRETS,QUAD LASER CANNON TURRETS,(VARIES ACCORDING TO SHIP MODEL),BUILD-IN SPACE BOMB.,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,DOUBLE BLASTER CANNON TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,HEAVY LASER CANNON TURRETS,DOUBLE LASER CANNONS,TWIN BLASTER CANNON TURRETS,LASER CANNONS TURRETS,HEAVY TRACTORBEAM PROJECTORS,HEAVY BLASTER CANNONS,DOUBLE HEAVY LASER TURRETS,QUAD TURBOLASER TURRETS
0,TIE Shuttle Craft,TIE Support Craft,TIE/sh Shuttle,18.0,7.80,1.0,2.0,17000.0,0.066667,90.0,28.0,http://insd.swcombine.com/insd/tieshut.htm,http://insd.swcombine.com/insd/Ship_files/Tieshut.jpg,-1.0,-1.0,-1.0,0.892039,0.028984,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TIE Experimental M3,TIE Experimental Craft,TIE/e3 Fighter,20.0,6.60,1.0,0.0,75.0,0.066667,102.0,16.0,http://insd.swcombine.com/insd/tieexm3.htm,http://insd.swcombine.com/insd/Ship_files/TIEEXM3.JPG,-1.0,-1.0,-1.0,0.892039,0.028984,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Nssis Class Chiss Clawcraft,Other Starfighters,Nssis Class Clawcraft,19.0,7.65,1.0,0.0,25.0,0.166667,92.0,18.0,http://insd.swcombine.com/insd/tiechiss.htm,http://insd.swcombine.com/insd/Ship_files/TIEchiss.jpg,1.5,20.0,-1.0,0.892039,0.028984,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Builder Shuttle,Landing Craft,Builder Shuttle Mark 1 and Mark 2,3.0,40.00,4.0,0.0,15000.0,0.222222,35.0,45.0,http://insd.swcombine.com/insd/builder.htm,http://insd.swcombine.com/insd/Ship_files/BUILDER.JPG,2.0,-1.0,2.0,1.244250,0.028984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Svelte Class Imperial Shuttle,Shuttles,Svelte Class Imperial Shuttle,24.0,17.00,6.0,15.0,50000.0,2.000000,60.0,53.0,http://insd.swcombine.com/insd/svelte.htm,http://insd.swcombine.com/insd/Ship_files/SVELTE.JPG,1.0,100.0,5.0,1.190873,0.028984,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [12]:
#@title Eliminar campos Name, Model y URLs

ndf = ndf.drop(columns=["Name", "Model", "URL_data", "URL_image"])

ndf.head()

,Ship Type,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft,LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LIGHT LASER CANNONS,MEDIUM LASER CANNONS,DOUBLE BLASTER CANNONS,QUAD LASER CANNONS,GRAVITY WELL PROJECTORS,ION CANNONS,PROTON TORPEDO LAUNCHERS,GENERAL PURPOSE LAUNCHERS,DOUBLE TURBOLASER CANNONS,TURBOLASER CANNONS,TRACTOR BEAM PROJECTORS,LASER CANNON TURRETS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,SUPERLASERS,MEDIUM ION CANNONS,TURBOLASER BATTERIES,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,TWIN LASER CANNONS,QUAD BLASTER CANNON,EMP CANNONS,MEDIUM TURBOLASERS,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,BLASTER CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,HULL-CUTTING AIRLOCKS,ION CANNON BATTERIES,ION CANNON TURRETS,REPEATING BLASTER TURRETS,QUAD TURBOLASER CANNONS,HEAVY TURBOLASER CANNONS,HEAVY TURBOLASER TURRETS,TURBOLASER TURRETS,DOUBLE LASER CANNON TURRETS,QUAD LASER CANNON TURRETS,(VARIES ACCORDING TO SHIP MODEL),BUILD-IN SPACE BOMB.,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,DOUBLE BLASTER CANNON TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,HEAVY LASER CANNON TURRETS,DOUBLE LASER CANNONS,TWIN BLASTER CANNON TURRETS,LASER CANNONS TURRETS,HEAVY TRACTORBEAM PROJECTORS,HEAVY BLASTER CANNONS,DOUBLE HEAVY LASER TURRETS,QUAD TURBOLASER TURRETS
0,TIE Support Craft,18.0,7.80,1.0,2.0,17000.0,0.066667,90.0,28.0,-1.0,-1.0,-1.0,0.892039,0.028984,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TIE Experimental Craft,20.0,6.60,1.0,0.0,75.0,0.066667,102.0,16.0,-1.0,-1.0,-1.0,0.892039,0.028984,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Other Starfighters,19.0,7.65,1.0,0.0,25.0,0.166667,92.0,18.0,1.5,20.0,-1.0,0.892039,0.028984,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Landing Craft,3.0,40.00,4.0,0.0,15000.0,0.222222,35.0,45.0,2.0,-1.0,2.0,1.244250,0.028984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Shuttles,24.0,17.00,6.0,15.0,50000.0,2.000000,60.0,53.0,1.0,100.0,5.0,1.190873,0.028984,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#@title Reordena columnas para que Ship Type quede al final

ndf = ndf.reindex(columns=(list([a for a in ndf.columns if a != 'Ship Type'] + ['Ship Type']) ))

ndf.head()

,Manufacturer,Length,Crew,Troops,Cargo Capacity,Consumables,Speed,Hull,Hyperdrive Multiplier,Shields,Hyperdrive Backup,Special Features,Onboard Craft,LASER CANNONS,CONCUSSION MISSILE LAUNCHERS,LIGHT LASER CANNONS,MEDIUM LASER CANNONS,DOUBLE BLASTER CANNONS,QUAD LASER CANNONS,GRAVITY WELL PROJECTORS,ION CANNONS,PROTON TORPEDO LAUNCHERS,GENERAL PURPOSE LAUNCHERS,DOUBLE TURBOLASER CANNONS,TURBOLASER CANNONS,TRACTOR BEAM PROJECTORS,LASER CANNON TURRETS,HEAVY TURBOLASER BATTERIES,HEAVY LASER CANNONS,SUPERLASERS,MEDIUM ION CANNONS,TURBOLASER BATTERIES,PROTON TORPEDOS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHERS.,TWIN LASER CANNONS,QUAD BLASTER CANNON,EMP CANNONS,MEDIUM TURBOLASERS,DOUBLE TURBOLASER TURRETS,TURBOLASER CANNON TURRETS,THERMAL DETONATOR LAUNCHERS,BLASTER CANNONS,ORBITAL MINE OR THERMAL DETONATOR LAUNCHER.,QUAD TURBOLASER BATTERIES,DOUBLE TURBOLASER BATTERIES,CUNCUSSION MISSILE LAUNCHERS,HULL-CUTTING AIRLOCKS,ION CANNON BATTERIES,ION CANNON TURRETS,REPEATING BLASTER TURRETS,QUAD TURBOLASER CANNONS,HEAVY TURBOLASER CANNONS,HEAVY TURBOLASER TURRETS,TURBOLASER TURRETS,DOUBLE LASER CANNON TURRETS,QUAD LASER CANNON TURRETS,(VARIES ACCORDING TO SHIP MODEL),BUILD-IN SPACE BOMB.,MEDIUM BLASTER CANNONS,LIGHT TURBOLASER CANNONS,HEAVY TURBOLASERS,MEDIUM DUAL TURBOLASERS,PROTON TORPEDO TUBES,DOUBLE BLASTER CANNON TURRETS,MU-1,LASER CANNONS / MU-3: 2 MEDIUM LASER CANNONS,HEAVY LASER CANNON TURRETS,DOUBLE LASER CANNONS,TWIN BLASTER CANNON TURRETS,LASER CANNONS TURRETS,HEAVY TRACTORBEAM PROJECTORS,HEAVY BLASTER CANNONS,DOUBLE HEAVY LASER TURRETS,QUAD TURBOLASER TURRETS,Ship Type
0,18.0,7.80,1.0,2.0,17000.0,0.066667,90.0,28.0,-1.0,-1.0,-1.0,0.892039,0.028984,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TIE Support Craft
1,20.0,6.60,1.0,0.0,75.0,0.066667,102.0,16.0,-1.0,-1.0,-1.0,0.892039,0.028984,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TIE Experimental Craft
2,19.0,7.65,1.0,0.0,25.0,0.166667,92.0,18.0,1.5,20.0,-1.0,0.892039,0.028984,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Other Starfighters
3,3.0,40.00,4.0,0.0,15000.0,0.222222,35.0,45.0,2.0,-1.0,2.0,1.244250,0.028984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Landing Craft
4,24.0,17.00,6.0,15.0,50000.0,2.000000,60.0,53.0,1.0,100.0,5.0,1.190873,0.028984,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Shuttles


In [14]:
#@title Mostrar estadísticas de datos preparados

generar_estadisticas_detalladas(ndf, "> Estadísticas de los datos preparados")



 > Estadísticas de los datos preparados : 


,Tipo Valor,Rango Valores,Promedio ± Desvío,¿Valores Ceros?,¿Valores Nulos?
Manufacturer,float64,[ 0.00 ; 29.00 ],14.626 ± 7.205,1,0
Length,float64,[ 2.10 ; 16000.00 ],481.423 ± 2123.802,0,0
Crew,float64,[ 0.00 ; 712645.00 ],14605.710 ± 90254.128,2,0
Troops,float64,[ 0.00 ; 150000.00 ],3293.393 ± 19163.088,50,0
Cargo Capacity,float64,[ -1.00 ; 600000000.00 ],15213112.766 ± 75767701.080,0,0
Consumables,float64,[ 0.03 ; 120.00 ],8.328 ± 19.252,0,0
Speed,float64,[ 4.00 ; 155.00 ],69.159 ± 38.796,0,0
Hull,float64,[ 5.00 ; 77710.00 ],1473.692 ± 8888.793,0,0
Hyperdrive Multiplier,float64,[ -1.00 ; 15.00 ],0.899 ± 2.004,0,0
Shields,float64,[ -1.00 ; 136000.00 ],2713.785 ± 15989.261,0,0


Tiene 75 atributos y 107 ejemplos.




#Exportar datos de Naves Imperiales

In [15]:
#@title Acceder al Drive

# monta Google Drive:
# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = '/content/gdrive/My Drive/IA/demoStarWars/datos/'  #@param {type:"string"}


Mounted at /content/gdrive


In [16]:
#@title Exporta los datos como CSV

download_CSV = False #@param{type:"boolean"}

import os

# crea directorio si no existe
if not os.path.isdir(path):
  os.mkdir(path)

def data2CSV(df, nomArch, descripcion):
  if df is None:
    print("No hay " + descripcion + " para exportar")
  df.to_csv(nomArch, index=False)
  if download_CSV:
    files.download(nomArch)
  print(descripcion + " exportados como ", nomArch)

# exporta datos
data2CSV(df, path + "navesOri.csv", "Datos originales")
data2CSV(ndf, path + "naves.csv", "Datos preparados")


Datos originales exportados como  /content/gdrive/My Drive/IA/demoStarWars/datos/navesOri.csv
Datos preparados exportados como  /content/gdrive/My Drive/IA/demoStarWars/datos/naves.csv
